In [ ]:
from selenium import webdriver
import datetime
import time
import winsound
now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
browser = webdriver.Chrome()

In [ ]:
amazon_link = "https://www.amazon.com/"
browser.get(amazon_link)

In [ ]:
def alert_beep(max_iter = 20):
    duration = 1000  # milliseconds
    freq = 440  # Hz
    for i in range(max_iter):
        winsound.Beep(freq, duration)
        print("beeping " + str(i))
        time.sleep(1)

In [ ]:
def check_schedule_page():
    schedule = browser.find_elements_by_xpath("//*[contains(text(),'Schedule your order')]")
    if len(schedule) < 1:
        print("Not on schedule page, login again")
        login(is_amazon_fresh = True)
        return False
    else:
        return True

In [ ]:
def login(alert = True, is_amazon_fresh = True):
    # alert_beep(5)
    try:
        browser.get(amazon_link)
        time.sleep(2)
        
        # cart
        browser.find_element_by_id("nav-cart").click()
        time.sleep(2)

        if is_amazon_fresh:
            # amamzon fresh continue
            browser.find_element_by_id("sc-alm-buy-box-ptc-button-QW1hem9uIEZyZXNo-announce").submit()
        else:
            # whole foods continue
            browser.find_element_by_id("sc-alm-buy-box-ptc-button-VUZHIFdob2xlIEZvb2Rz-announce").submit()
            time.sleep(2)

        # Before you check out:
        browser.find_element_by_id("a-autoid-0").click()
        time.sleep(2)
        
        if not is_amazon_fresh:
            # confirm subsitution continue, amazon fresh don't need this
            browser.find_element_by_id("subsContinueButton").click()
            time.sleep(2)
        assert check_schedule_page(), "This is not schedule your order page."
    except:
        print("Failed to login in again")
        if alert:
            alert_beep()

In [ ]:
login(alert = False)

In [ ]:
def find_string_on_web(target):
    xpath_str = "//*[contains(text(),'" + target + "')]"
    find_outputs = browser.find_elements_by_xpath(xpath_str)
    if len(find_outputs) != 0:
        print("Come to buy " + target + " window! ")
        return True
    return False

In [ ]:
end_time = datetime.datetime(2020, 4, 14, hour = 23, minute=30)
now = datetime.datetime.now()
count = 0
target_strings = ['0 AM',
                '0 PM',
                '1-hour',
                '2-hour']
while now < end_time:
    now = datetime.datetime.now()
    now_str = now.strftime('%Y-%m-%d %H:%M:%S.%f')
    browser.refresh()
    if check_schedule_page():
        is_find = False
        for s in target_strings:
            is_find |= find_string_on_web(s)
            if is_find and s == '0 PM' and find_string_on_web('Next available'):
                is_find = False               
        if is_find:
            alert_beep()
            break
        else:
            print("checked at " + now_str + " : " + str(count))
    time.sleep(5)
    count += 1
print(" code finished ")